## `LLM`

In [1]:
import os 
from dotenv import load_dotenv
from langchain_groq import ChatGroq

os.chdir('/Users/reejungkim/Documents/Git/working-in-progress')
load_dotenv()

# Step 1: Set up the LLM with Groq as the provider
# Available models: 'mixtral-8x7b-32768', 'llama2-70b-4096', etc.
llm = ChatGroq(
    model="llama-3.1-8b-instant",  # Choose a model supported by Groq
    temperature=0.7,             # Adjust as needed
    max_tokens=512,              # Adjust as needed
    groq_api_key=os.environ["groq_api"] 
)

In [42]:
llm.invoke("what is gemini")

AIMessage(content="Gemini can refer to several things:\n\n1. **Gemini (Constellation)**: Gemini is a constellation in the northern sky, named after the twins in Greek mythology. It is one of the 12 constellations of the zodiac and is associated with the mythological story of Castor and Pollux.\n\n2. **Gemini (Zodiac Sign)**: Gemini is a zodiac sign, associated with the dates May 21 - June 20. People born under this sign are said to be adaptable, communicative, and quick-witted.\n\n3. **Gemini (AI Assistant)**: Gemini is also the name of a conversational AI assistant developed by Google. It is a large language model that can engage in natural-sounding conversations and answer a wide range of questions.\n\n4. **Gemini (Spacecraft)**: Gemini was a NASA space program in the 1960s, which aimed to develop a spacecraft capable of carrying astronauts into space for extended periods. The Gemini program played an essential role in the development of the Apollo program.\n\nThese are a few example

## `Tool` 

In [ ]:
import os
load_dotenv()

from langchain_community.tools.tavily_search import TavilySearchResults
search = TavilySearchResults(tavily_api_key=os.environ['tavily_api'])

In [6]:
tools = [search]

In [7]:
search.invoke({"query": "What happened at the last wimbledon"})

[{'title': "2025 Wimbledon Championships – Men's singles - Wikipedia",
  'url': 'https://en.wikipedia.org/wiki/2025_Wimbledon_Championships_%E2%80%93_Men%27s_singles',
  'content': '23. ^ Benson, Pat (3 June 2025). "Nick Kyrgios Pulls Out of Wimbledon After "Small Setback"". Sports Illustrated. Retrieved 4 June 2025.\n24. ^ "This is the important last minute withdrawal at Wimbledon 2025". puntodebreak.com. Retrieved 27 June 2025.\n25. ^ "Injured Hurkacz withdraws from Wimbledon". Reuters. 27 June 2025. Retrieved 27 June 2025. [...] 16. ^  "Record-Setting Carnage at Wimbledon as Numerous Top Seeds Exit Early". SI.com. 2 July 2025.\n17. ^  "Fognini announces retirement". ATPTour. 9 July 2025.\n18. ^ a b  "Lingering knee injury forces Casper Ruud to pull out of Wimbledon". ESPN. 21 June 2025.\n19. ^ a b "Casper Ruud, Arthur Fils to miss Wimbledon with ongoing injuries". tennis.com. Retrieved 27 June 2025. [...] 1. ^ "Alcaraz crushes Djokovic to retain Wimbledon title". BBC Sport. 14 July 

## `Prompt` 

In [26]:
# https://smith.langchain.com/hub/rlm/rag-prompt
# # https://smith.langchain.com/hub 에서 원하는 prompt를 받아올 수 있습니다.

from langchain import hub
# 프롬프트를 가져오기 위해 API 키 설정
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
prompt = hub.pull("hwchase17/openai-functions-agent")


/opt/anaconda3/envs/env_llm/lib/python3.10/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [ ]:
prompt.

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='

In [31]:
from langchain.agents import create_tool_calling_agent  
from langchain.agents import AgentExecutor  

agent = create_tool_calling_agent(llm, tools, prompt)  
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)  

In [32]:
agent_executor.invoke({"input":"What is the mission of DASA"})



> Entering new AgentExecutor chain...
<function=t

> Finished chain.


{'input': 'What is the mission of DASA', 'output': '<function=t'}

In [34]:
agent_executor.invoke({"input":"What is weather in Kuala Lumpur?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'Kuala Lumpur weather'}`


HTTPError('401 Client Error: Unauthorized for url: https://api.tavily.com/search')

APIError: tool call validation failed: attempted to call tool 'brave_search' which was not in request.tools

In [41]:
from langchain_tavily import TavilySearch
from langgraph.prebuilt import create_react_agent

# Initialize Tavily Search Tool
tavily_search_tool = TavilySearch(
    max_results=5,
    topic="general",
)

agent = create_react_agent(llm, [tavily_search_tool])

user_input = "What nation hosted the Euro 2024? Include only wikipedia sources."

for step in agent.stream(
    {"messages": user_input},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

ModuleNotFoundError: No module named 'langchain_tavily'